In [4]:
from typing import Callable, Any
import numpy as np
from scipy.stats import norm, truncnorm

In [5]:
def order_variable(Sigma, a, orderer=np.argmin, debug=False):
    # Tested
    """
    Returns an ordered version of the orthant probability problem {P(X >=a), X ~ N(0, Sigma)}.
    """
    d = len(Sigma)
    obj = _VariableOrdering(mu=np.array([0] * d), Sigma=Sigma.copy(), a=a.copy(), dummy=np.arange(d), orderer=orderer, debug=debug)
    new_order = [i for i in obj]
    if debug:
        print(new_order)
    assert len(set(new_order)) == d
    return Sigma[np.ix_(new_order, new_order)], a[new_order]


In [6]:
class _VariableOrdering:
    # eye-tested
    def __init__(self, mu, Sigma, a, dummy, orderer, debug=False):
        self.mu, self.Sigma, self.a, self.dummy, self.orderer = mu, Sigma, a, dummy, orderer
        self.debug = debug

    def __iter__(self):
        return self

    def __next__(self):
        if len(self.dummy) < 1:
            raise StopIteration
        probas = norm.logsf(self.a, loc=self.mu, scale=np.sqrt(np.diag(self.Sigma)))
        chosen_index = self.orderer(probas)
        condition_value = self.__class__.truncnorm_mean(mean=self.mu[chosen_index], var=self.Sigma[chosen_index, chosen_index], a=self.a[chosen_index])
        [self.__class__.reorder_1d(x, chosen_index) for x in [self.mu, self.a, self.dummy]]
        self.__class__.reorder_2d(self.Sigma, chosen_index)
        self.mu, self.Sigma = self.__class__.condition_on_first_variable(mu=self.mu, Sigma=self.Sigma, val=condition_value)
        if self.debug:
            print(self.mu)
            print(self.Sigma)
        self.a = self.a[1:]
        res = self.dummy[0]
        self.dummy = self.dummy[1:]
        return res

    @staticmethod
    def reorder_1d(x,i):
        x[i], x[0] = x[0], x[i]

    @staticmethod
    def reorder_2d(x, i):
        #tested
        x[i,:], x[0,:] = x[0,:].copy(), x[i,:].copy()
        x[:,i], x[:,0] = x[:,0].copy(), x[:,i].copy()

    @staticmethod
    def condition_on_first_variable(mu, Sigma, val):
        # tested
        """
        :param mu: numpy array of size (d,)
        :param Sigma: numpy array of size (d,d)
        :param val: float
        :return: mu_prime, Sigma_prime: mean and covariance matrix of N(X_{1:d-1} | X_0 = val), where X_{0:d-1} ~ N(mu, Sigma)
        """
        mu_prime = Sigma[0, :] / Sigma[0, 0] * (val - mu[0]) + mu
        Sigma_prime = Sigma - np.outer(Sigma[0, :], Sigma[0, :]) / Sigma[0, 0]
        return mu_prime[1:], Sigma_prime[1:, 1:]

    @staticmethod
    def truncnorm_mean(mean, var, a):
        # tested
        """
        Calculate the mean of N(mean, var) conditionnally on x >=a.
        """
        s = var ** 0.5
        return mean + s * truncnorm.moment(1, a=(a-mean)/s, b=np.inf)


In [12]:
a = np.load("a_rc150.npy")
S = np.load("Sigma_rc150.npy")

In [13]:
newS,newa = order_variable(S,a)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])